In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHFCzFO7YkWrCbAWB8RrKu6UOD4Bv1z9X3Gvh1Xx8cKSVNjyeCVN7E
Mounted at /content/drive


In [10]:
import pandas as pd

In [11]:
df_OH = pd.read_csv('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/01.운영이력.csv') 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# 시간을 계산하기 위해 'LEAS_DATE','RTN_DATE'두개의 컬럼을 '년월일시분초' 형식으로 변경하여 준다 
df_OH['LEAS_DATE'] =  pd.to_datetime(df_OH['LEAS_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
df_OH['RTN_DATE'] =  pd.to_datetime(df_OH['RTN_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [13]:
# 'RTN_DATE' - 'LEAS_DATE 를 통해 새로운 컬럼 초단위를 기준으로 하는 'Biek_Time(s)' 컬럼을 만들어 준다.
df_OH['Bike_Time(s)'] = df_OH['RTN_DATE'] - df_OH['LEAS_DATE'] # 이용 시간 Column 추가
df_OH['Bike_Time(s)'] = df_OH['Bike_Time(s)'].dt.total_seconds() #초단위로 나타내 준다.

In [14]:
del df_OH['LEAS_STAT'] # 설명 안되는 데이터 프레임 제거

In [15]:
# 컬럼 목록을 한글로 정리
df_OH.columns = ['대여 번호', '대여 시간', '대여 스테이션 번호', '대여 거치대 번호', '반납 시간','반납 스테이션 번호','반납 거치대 번호','추정 이동 거리', '회원 구분', '회원 번호', '비회원 번호', '자전거 번호', '관제 반납','이용 시간(s)']

In [16]:
station = df_OH 

In [17]:
station = station.fillna(0)

In [18]:
import datetime as dt

station['대여 시간']= pd.to_datetime(station['대여 시간']) 
station['대여 시간'] = station['대여 시간'].dt.strftime('%Y-%m')
station['반납 시간'] = pd.to_datetime(station['반납 시간'])
station['반납 시간'] = station['반납 시간'].dt.strftime('%Y-%m')
station.head()

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01,213,18,2017-01,260,17,0.0,6,164203,0.0,1A844000000BB7,0.0,791.0
1,15945542,2017-01,231,4,2017-01,231,17,31039.0,1,187551,0.0,1A844000000494,0.0,2961.0
2,15945543,2017-01,119,14,2017-01,0,0,0.0,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01,121,17,2017-01,133,14,15490.0,12,183971,0.0,1A844000000731,0.0,829.0
4,15945545,2017-01,320,29,2017-01,259,27,0.0,12,167475,0.0,1A84400000F343,0.0,912.0


#1. 반납 스테이션 0일때 각 대여 스테이션에서의 대여 횟수

In [19]:
total_number = station.groupby(['대여 스테이션 번호', '반납 스테이션 번호']).size()
total_number = total_number.to_frame(name = '이용 횟수').reset_index()
total_number.head()


,대여 스테이션 번호,반납 스테이션 번호,이용 횟수
0,101,0,1115
1,101,101,1187
2,101,103,336
3,101,105,94
4,101,106,213


In [20]:
return_zero = total_number.loc[total_number['반납 스테이션 번호'] == 0].reset_index().drop(['index'], axis=1)
return_zero.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수
0,101,0,1115
1,103,0,792
2,104,0,1845
3,105,0,599
4,106,0,757


# 2. 반납 스테이션 번호 0일때 평균 추정 이동 거리

In [21]:
Station = station[['대여 시간', '대여 스테이션 번호', '반납 시간', '반납 스테이션 번호', '추정 이동 거리']]
distance = Station.groupby(['대여 스테이션 번호', '반납 스테이션 번호']).sum().reset_index()
distance_return_zero = distance.loc[distance['반납 스테이션 번호'] == 0].reset_index().drop(['index'], axis=1)
distance_return_zero.head()

,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리
0,101,0,82689.0
1,103,0,71720.0
2,104,0,98118.0
3,105,0,0.0
4,106,0,0.0


In [22]:
#평균 추정 이동 거리를 구해야하므로, 추정 이동 거리가 0인 경우는 제외하고 이용 횟수를 다시 구함.
station_nonNaN = station[(station['추정 이동 거리'] != 0)]
station_nonNaN= station_nonNaN.reset_index()
station_nonNaN = station_nonNaN.drop(['index'], axis=1)
station_nonNaN.head()

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945542,2017-01,231,4,2017-01,231,17,31039.0,1,187551,0.0,1A844000000494,0.0,2961.0
1,15945544,2017-01,121,17,2017-01,133,14,15490.0,12,183971,0.0,1A844000000731,0.0,829.0
2,15945547,2017-01,232,7,2017-01,213,18,8948.0,99,0,3185082.0,1A84400000F57C,0.0,541.0
3,15945550,2017-01,114,8,2017-01,113,13,17482.0,12,138642,0.0,1A84400000F113,0.0,164.0
4,15945554,2017-01,215,2,2017-01,231,19,15170.0,2,187342,0.0,1A84400000F09D,0.0,852.0


In [23]:
station_nonNaN['대여 시간']= pd.to_datetime(station_nonNaN['대여 시간']) 
station_nonNaN['대여 시간'] = station_nonNaN['대여 시간'].dt.strftime('%Y-%m')
station_nonNaN['반납 시간'] = pd.to_datetime(station_nonNaN['반납 시간'])
station_nonNaN['반납 시간'] = station_nonNaN['반납 시간'].dt.strftime('%Y-%m')

In [24]:
total_number_nonNaN = station_nonNaN.groupby(['대여 스테이션 번호', '반납 스테이션 번호']).size()
total_number_nonNaN = total_number_nonNaN.to_frame(name = '이용 횟수').reset_index()
total_number_nonNaN.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수
0,101,0,3
1,101,101,472
2,101,103,143
3,101,105,34
4,101,106,124


In [25]:
return_zero_nonNaN = total_number_nonNaN.loc[total_number_nonNaN['반납 스테이션 번호'] == 0].reset_index().drop(['index'], axis=1)
return_zero_nonNaN.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수
0,101,0,3
1,103,0,2
2,104,0,3
3,110,0,4
4,111,0,3


In [26]:
distance_return_zero['1회당 추정 이동 거리'] = ''
for i in range(len(distance_return_zero)):
  for j in range(len(return_zero_nonNaN)):
    if distance_return_zero['대여 스테이션 번호'][i] == return_zero_nonNaN['대여 스테이션 번호'][j]:
      distance_return_zero['1회당 추정 이동 거리'][i] = distance_return_zero['추정 이동 거리'][i]/return_zero_nonNaN['이용 횟수'][j]
distance_return_zero.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,대여 스테이션 번호,반납 스테이션 번호,추정 이동 거리,1회당 추정 이동 거리
0,101,0,82689.0,27563
1,103,0,71720.0,35860
2,104,0,98118.0,32706
3,105,0,0.0,
4,106,0,0.0,


#3. 회원/비회원 간 대여 0 비교

#4. 전체 이용 횟수 대비 반납 0 이용 횟수

In [27]:
total = total_number.groupby(['대여 스테이션 번호']).sum().reset_index()
total = total[['대여 스테이션 번호', '이용 횟수']]
total.head()

,대여 스테이션 번호,이용 횟수
0,101,8366
1,103,4535
2,104,13402
3,105,2806
4,106,4444


# 5. 전부 merge 해주기

In [28]:
return_zero = return_zero.merge(distance_return_zero)
return_zero.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수,추정 이동 거리,1회당 추정 이동 거리
0,101,0,1115,82689.0,27563
1,103,0,792,71720.0,35860
2,104,0,1845,98118.0,32706
3,105,0,599,0.0,
4,106,0,757,0.0,


In [29]:
return_zero['반납 0 이용 횟수/전체 이용 횟수'] = ''
for i in range(len(return_zero)):
  return_zero['반납 0 이용 횟수/전체 이용 횟수'][i] = return_zero['이용 횟수'][i]/total['이용 횟수'][i]
return_zero.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,대여 스테이션 번호,반납 스테이션 번호,이용 횟수,추정 이동 거리,1회당 추정 이동 거리,반납 0 이용 횟수/전체 이용 횟수
0,101,0,1115,82689.0,27563,0.133278
1,103,0,792,71720.0,35860,0.174642
2,104,0,1845,98118.0,32706,0.137666
3,105,0,599,0.0,,0.213471
4,106,0,757,0.0,,0.170342


#6. 대여 스테이션 좌표 추가 

In [30]:
bike_station = pd.read_csv('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/02.자전거스테이션.csv', index_col = 0)
bike_station = bike_station.reset_index()
bike_station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530


In [31]:
return_zero['대여 위도'] = ''
return_zero['대여 경도'] = ''
for i in range(len(return_zero)):
  for j in range(len(bike_station)):
    if return_zero['대여 스테이션 번호'][i] == bike_station['Station_ID'][j]:
      return_zero['대여 위도'][i] = format(bike_station['위도'][j], ".6f")
      return_zero['대여 경도'][i] = format(bike_station['경도'][j], ".6f")

return_zero.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,대여 스테이션 번호,반납 스테이션 번호,이용 횟수,추정 이동 거리,1회당 추정 이동 거리,반납 0 이용 횟수/전체 이용 횟수,대여 위도,대여 경도
0,101,0,1115,82689.0,27563,0.133278,37.654775,126.834584
1,103,0,792,71720.0,35860,0.174642,37.660442,126.840377
2,104,0,1845,98118.0,32706,0.137666,37.698523,126.766042
3,105,0,599,0.0,,0.213471,37.655244,126.839261
4,106,0,757,0.0,,0.170342,37.653410,126.842530


In [32]:
return_zero['대여 위도'] = pd.to_numeric(return_zero['대여 위도'], errors='coerce')
return_zero['대여 경도'] = pd.to_numeric(return_zero['대여 경도'], errors='coerce')

In [33]:
return_zero = return_zero.fillna(0)

# 사분위수로 반납 스테이션 0 나눠서 시각화

In [34]:
import pandas as pd
!pip install geopandas
import geopandas as gpd
!pip install deckgljupyter
import deckgljupyter.Layer as deckgl
import folium

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 14.8MB 318kB/s 
     |████████████████████████████████| 10.9MB 35.9MB/s 
     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for chroma-py: filename=chroma_py-0.1.0.dev1-cp36-none-any.whl size=5107 sha256=e21d8350c41f83f6ff680408d7b6c7a162eeedb2472a1cf6fdae2338ce63b75e
  Stored in directory: /root/.cache/pip/wheels/43/3b/8c/3f6d7536b8bef26b7c3be5989f8103513eb949e50a4f9f81cf
Successfully built chroma-py


In [35]:
return_zero_1st = return_zero.loc[return_zero['반납 0 이용 횟수/전체 이용 횟수'] <= return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.25)]
return_zero_2nd = return_zero.loc[(return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.25) <= return_zero['반납 0 이용 횟수/전체 이용 횟수']) & (return_zero['반납 0 이용 횟수/전체 이용 횟수']  < return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.5))]
return_zero_3rd = return_zero.loc[(return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.5) <= return_zero['반납 0 이용 횟수/전체 이용 횟수']) & (return_zero['반납 0 이용 횟수/전체 이용 횟수']  < return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.75))]
return_zero_rest = return_zero.loc[return_zero['반납 0 이용 횟수/전체 이용 횟수'].quantile(.75) <= return_zero['반납 0 이용 횟수/전체 이용 횟수']]

In [36]:
border=gpd.read_file('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/09.행정경계(읍면동).geojson')
border['population'] = 0

population=gpd.read_file('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/06.인구(거주)분포도(100M X 100M).geojson')
population = population.fillna(0)

for i in range(len(border)):
  for j in range(len(population)):
    if border['geometry'][i].contains(population['geometry'][j]):
      border['population'][i] += population['val'][j]

import copy
border_2 = copy.deepcopy(border)

import folium
center = (37.647972, 126.835398); zoom = 13
n = folium.Map(location=center, zoom_start=zoom)
folium.Choropleth(
    geo_data = border,
    data = border_2,
    columns = ('EMD_KOR_NM', 'population'),
    key_on = 'feature.properties.EMD_KOR_NM',
    fill_color = 'BuPu'
).add_to(n)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
for index, row in return_zero_1st.iterrows():
    folium.CircleMarker([row['대여 위도'], row['대여 경도']], color = 'white', radius = 0.002 * row['이용 횟수']).add_to(n)
for index, row in return_zero_2nd.iterrows():
    folium.CircleMarker([row['대여 위도'], row['대여 경도']], color = 'orange', radius = 0.002 * row['이용 횟수']).add_to(n)
for index, row in return_zero_3rd.iterrows():
    folium.CircleMarker([row['대여 위도'], row['대여 경도']], color = 'green', radius = 0.002 * row['이용 횟수']).add_to(n)
for index, row in return_zero_rest.iterrows():
    folium.CircleMarker([row['대여 위도'], row['대여 경도']], color = 'black', radius = 0.002 * row['이용 횟수']).add_to(n)
n

#원 색깔: (특정 스테이션에서의) 관제 반납 횟수/전체 이용 횟수
#원 크기: 관제 반납 횟수

# 동 단위 반납 스테이션 0인 스테이션의 반납 0 이용 횟수/전체 이용 횟수 비율

In [38]:
border=gpd.read_file('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/09.행정경계(읍면동).geojson')
border.head()

,EMD_CD,EMD_KOR_NM,geometry
0,41281101,주교동,"MULTIPOLYGON (((126.81068 37.65820, 126.81069 ..."
1,41281102,원당동,"MULTIPOLYGON (((126.83321 37.68013, 126.83340 ..."
2,41281103,신원동,"MULTIPOLYGON (((126.86362 37.67729, 126.86364 ..."
3,41281104,원흥동,"MULTIPOLYGON (((126.85975 37.65081, 126.85990 ..."
4,41281105,도내동,"MULTIPOLYGON (((126.84806 37.62348, 126.84835 ..."


In [39]:
return_zero.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수,추정 이동 거리,1회당 추정 이동 거리,반납 0 이용 횟수/전체 이용 횟수,대여 위도,대여 경도
0,101,0,1115,82689.0,27563,0.133278,37.654775,126.834584
1,103,0,792,71720.0,35860,0.174642,37.660442,126.840377
2,104,0,1845,98118.0,32706,0.137666,37.698523,126.766042
3,105,0,599,0.0,,0.213471,37.655244,126.839261
4,106,0,757,0.0,,0.170342,37.653410,126.842530


In [40]:
from geopandas import points_from_xy
return_zero = gpd.GeoDataFrame(return_zero, geometry=points_from_xy(return_zero['대여 경도'], return_zero['대여 위도']))
return_zero.head()

,대여 스테이션 번호,반납 스테이션 번호,이용 횟수,추정 이동 거리,1회당 추정 이동 거리,반납 0 이용 횟수/전체 이용 횟수,대여 위도,대여 경도,geometry
0,101,0,1115,82689.0,27563,0.133278,37.654775,126.834584,POINT (126.83458 37.65478)
1,103,0,792,71720.0,35860,0.174642,37.660442,126.840377,POINT (126.84038 37.66044)
2,104,0,1845,98118.0,32706,0.137666,37.698523,126.766042,POINT (126.76604 37.69852)
3,105,0,599,0.0,,0.213471,37.655244,126.839261,POINT (126.83926 37.65524)
4,106,0,757,0.0,,0.170342,37.653410,126.842530,POINT (126.84253 37.65341)


In [41]:
border['반납0/전체'] = ''
for i in range(len(border)):
  count = 0
  ratio = 0
  for j in range(len(return_zero)):
    if border['geometry'][i].contains(return_zero['geometry'][j]):
      count += 1
      ratio += return_zero['반납 0 이용 횟수/전체 이용 횟수'][j]
  if count == 0:
    border['반납0/전체'][i] = 0
  else:
    border['반납0/전체'][i] = ratio/count

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [42]:
border

,EMD_CD,EMD_KOR_NM,geometry,반납0/전체
0,41281101,주교동,"MULTIPOLYGON (((126.81068 37.65820, 126.81069 ...",0.160719
1,41281102,원당동,"MULTIPOLYGON (((126.83321 37.68013, 126.83340 ...",0
2,41281103,신원동,"MULTIPOLYGON (((126.86362 37.67729, 126.86364 ...",0.125837
3,41281104,원흥동,"MULTIPOLYGON (((126.85975 37.65081, 126.85990 ...",0.129211
4,41281105,도내동,"MULTIPOLYGON (((126.84806 37.62348, 126.84835 ...",0.216924
5,41281106,성사동,"MULTIPOLYGON (((126.83047 37.64860, 126.83053 ...",0.172933
6,41281107,북한동,"MULTIPOLYGON (((126.95320 37.65582, 126.95334 ...",0
7,41281108,효자동,"MULTIPOLYGON (((126.94756 37.65922, 126.94777 ...",0
8,41281109,지축동,"MULTIPOLYGON (((126.89619 37.65389, 126.89627 ...",0
9,41281110,오금동,"MULTIPOLYGON (((126.89197 37.67066, 126.89222 ...",0


#동 별 반납0 횟수/전체 대여 횟수와 동 별 스테이션 간 거리의 상관관계 분석

In [43]:
bike_station_dong = pd.read_csv('/content/drive/My Drive/프로젝트/compas/동명씨 학술자료/인구분석/동별_전체평균_500평균_스테이션 갯수.csv', index_col=0)
bike_station_dong.head()

,EMD_KOR_NM,harv(m)_mean,harv(m)_Less than 500,Count
0,성사동,614.160193,383.862977,4
1,탄현동,680.163285,378.576435,8
2,화정동,790.823164,320.174853,13
3,토당동,646.269375,477.422658,4
4,행신동,896.254871,401.809122,13


In [44]:
border_dong = border.merge(bike_station_dong)
border_dong = border_dong[['반납0/전체', 'harv(m)_Less than 500', 'Count']]
border_dong = border_dong.fillna(0)
border_dong = border_dong.rename({'harv(m)_Less than 500':'행정동 내 스테이션 간 거리(500m 이하)', 'Count':'행정동 내 스테이션 수'}, axis=1)

border_corr = border_dong.corr()                          #스테이션 간 거리가 가깝다고 해도 관제 반납의 비율에 유의미한 상관관계는 없는것으로 보임
border_corr.to_csv('border_corr.csv')

In [45]:
border_corr

,반납0/전체,행정동 내 스테이션 간 거리(500m 이하),행정동 내 스테이션 수
반납0/전체,1.000000,0.103410,0.096355
행정동 내 스테이션 간 거리(500m 이하),0.103410,1.000000,0.628329
행정동 내 스테이션 수,0.096355,0.628329,1.000000
